<a href="https://colab.research.google.com/github/whiteribbonalliance/womenshealthandwellbeing_public/blob/main/Inference_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 46.4 MB

**Upload you file for which you need predictions**

In [2]:
from google.colab import files
uploaded = files.upload()

Saving sample pred file.xlsx to sample pred file.xlsx


In [14]:
import pandas as pd
import os

def load_file_into_dataframe(file_path):
    # Get the file extension
    _, file_extension = os.path.splitext(file_path)

    # Load the file based on its extension
    if file_extension in ['.csv']:
        df = pd.read_csv(file_path)
    elif file_extension in ['.xls', '.xlsx']:
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format")

    return df

file_path = list(uploaded.keys())[0]     ####finename of the upload file
df = load_file_into_dataframe(file_path)

df.head()

,ingestion_time,respondent_age,respondent_age_bucket,respondent_country_code,respondent_region_name,respondent_gender,response_original_text,response_english_text,response_nlu_category,response_nlu_confidence,country_name,english_text,Top3_Predictions,Top3_Confidences,Multiple_Demands
0,2023-08-06 15:55:22.659102+00:00,59,55-64,GH,Ahafo,woman,security,NaN,"STI Information, Services and Supplies",0.998518,Ghana,security,"['No Harmful Practices and GBV', 'No Discrimin...","[0.00018367964660596003, 0.0002037123223752922...",False
1,2023-08-06 15:55:22.659102+00:00,23,20-24,KE,Makueni,woman,I want scan of cancer,NaN,"Breast, Cervical and Other Cancers",0.999340,Kenya,I want scan of cancer,"['Renewable Energy', 'Sexual and Reproductive ...","[0.00013826837054146073, 0.0001878863268825913...",False
2,2023-08-06 15:55:22.659102+00:00,19,15-19,KE,Meru,woman,Cheap cancer treatment,NaN,"Breast, Cervical and Other Cancers",0.999315,Kenya,Cheap cancer treatment,"['Sexual and Reproductive Health', 'Geriatric ...","[0.00017684416307210446, 0.0002093932834365211...",False
3,2023-08-06 15:55:22.659102+00:00,16,15-19,KE,Turkana,woman,I want to practise self lifting techniques to ...,NaN,Physical Activity,0.998564,Kenya,I want to practise self lifting techniques to ...,"['Physical Activity', 'Non-Communicable Diseas...","[0.0013179419028866615, 0.0014645383968682792,...",False
4,2023-08-06 15:55:22.659102+00:00,23,20-24,KE,Kiambu,woman,Screening of breast cancer for free,NaN,"Breast, Cervical and Other Cancers",0.999238,Kenya,Screening of breast cancer for free,"['Geriatric Health', 'Sexual and Reproductive ...","[0.0001503832175775038, 0.00017530146211441587...",False


In [15]:
### define the text column name here

response_column_name = 'english_text'

df['text_lower'] = df [response_column_name].str.lower()

In [11]:
list(uploaded.keys())[0]

'sample pred file.xlsx'

In [12]:
from simpletransformers.classification import ClassificationModel, MultiLabelClassificationModel
from scipy.special import softmax
from sklearn.preprocessing import LabelBinarizer

saved_model_dir = "amoldwalunj/BERT_multi_label_classification_survey_repose_classification"

# Load the model from the checkpoint
model = MultiLabelClassificationModel("bert", saved_model_dir, use_cuda=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [13]:
index_to_label= {0: 'Autonomy, Equality, and Empowerment',
 1: 'Cancer',
 2: 'Communicable Diseases',
 3: 'Education and Vocational Skills',
 4: 'Food and Adequate Nutrition',
 5: 'Free, Affordable or Insured Healthcare',
 6: 'Fully-functional and well-equipped Health Facilities',
 7: 'Health Workers',
 8: 'High Quality, Inclusive and Respectful Health Services',
 9: 'Interpersonal Relationships and Social Support',
 10: 'Maternal, Newborn, Child Health',
 11: 'Menstrual & Menopausal Cycles',
 12: 'Mental Health',
 13: 'No Harmful Practices and GBV',
 14: 'Non-Communicable Diseases',
 15: 'Physical Activity and Rest',
 16: 'Policy and Social Welfare',
 17: 'Safety',
 18: 'Sexual and Reproductive Health',
 19: 'Sustainable Energy & Agriculture',
 20: 'Transportation and Road Safety',
 21: 'Water, Sanitation & Hygiene',
 22: 'Work & Financial Support'}

In [17]:
# Create a dictionary mapping from index to class name
#index_to_label = {i: label for i, label in enumerate(mlb.classes_)}

def labels_from_predictions(predictions):
    """Convert predictions to label names using the index_to_label dictionary."""
    return [[index_to_label[idx] for idx, val in enumerate(row) if val == 1] for row in predictions]

def predict_single(text):
    """Predict the labels for a single text input."""
    predictions, _ = model.predict([text])
    return labels_from_predictions(predictions)[0]

def predict_batch(texts):
    """Predict the labels for a batch of text inputs."""
    predictions, _ = model.predict(texts)
    return labels_from_predictions(predictions)


def predict_dataframe(df):
    """Predict the labels for an entire dataframe and add them to a new 'demands' column."""
    texts = df['text_lower'].tolist()
    predictions, _ = model.predict(texts)
    df['demands'] = labels_from_predictions(predictions)
    return df


In [18]:
df= predict_dataframe(df)

  0%|          | 0/315 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [19]:
df.head()

,ingestion_time,respondent_age,respondent_age_bucket,respondent_country_code,respondent_region_name,respondent_gender,response_original_text,response_english_text,response_nlu_category,response_nlu_confidence,country_name,english_text,Top3_Predictions,Top3_Confidences,Multiple_Demands,text_lower,demands
0,2023-08-06 15:55:22.659102+00:00,59,55-64,GH,Ahafo,woman,security,NaN,"STI Information, Services and Supplies",0.998518,Ghana,security,"['No Harmful Practices and GBV', 'No Discrimin...","[0.00018367964660596003, 0.0002037123223752922...",False,security,[Safety]
1,2023-08-06 15:55:22.659102+00:00,23,20-24,KE,Makueni,woman,I want scan of cancer,NaN,"Breast, Cervical and Other Cancers",0.999340,Kenya,I want scan of cancer,"['Renewable Energy', 'Sexual and Reproductive ...","[0.00013826837054146073, 0.0001878863268825913...",False,i want scan of cancer,[Cancer]
2,2023-08-06 15:55:22.659102+00:00,19,15-19,KE,Meru,woman,Cheap cancer treatment,NaN,"Breast, Cervical and Other Cancers",0.999315,Kenya,Cheap cancer treatment,"['Sexual and Reproductive Health', 'Geriatric ...","[0.00017684416307210446, 0.0002093932834365211...",False,cheap cancer treatment,"[Cancer, Free, Affordable or Insured Healthcare]"
3,2023-08-06 15:55:22.659102+00:00,16,15-19,KE,Turkana,woman,I want to practise self lifting techniques to ...,NaN,Physical Activity,0.998564,Kenya,I want to practise self lifting techniques to ...,"['Physical Activity', 'Non-Communicable Diseas...","[0.0013179419028866615, 0.0014645383968682792,...",False,i want to practise self lifting techniques to ...,[Physical Activity and Rest]
4,2023-08-06 15:55:22.659102+00:00,23,20-24,KE,Kiambu,woman,Screening of breast cancer for free,NaN,"Breast, Cervical and Other Cancers",0.999238,Kenya,Screening of breast cancer for free,"['Geriatric Health', 'Sexual and Reproductive ...","[0.0001503832175775038, 0.00017530146211441587...",False,screening of breast cancer for free,"[Cancer, Free, Affordable or Insured Healthcare]"


In [20]:
df.to_csv('final_predictions.csv')